In [ ]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from openai import OpenAI
from tqdm import tqdm

In [ ]:
os.environ["OPENAI_API_KEY"] = ""


def get_response(msg):
    messages = [
        {
            "role": "user",
            "content": msg
        }
    ]

    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=messages,
        temperature=0.9
    )

    output = response.choices[0].message.content
    return output

In [ ]:
stage1_prompt = """
Brainstorm a list of potentially useful text rewritten tasks. Here are a few examples for your reference: 
- Adapt writing style to imitate a specific author or genre. 
- Rewrite text to be more formal, suitable for professional or academic settings.
- Convert this into a sea shanty.
Please adhere to the following guidelines: 
- Specify what the task is, and what the rewritten text are. 
- Each rewritten task should cover a wide range of queries, and should not be too specific. 
Your output should always be a python list of strings only, with about 50 elements, and each element corresponds to a distinct rewritten task in one sentence.

Here is an output format example:
- xxxxxxxxxx.
- xxxxxxxx.

Do not explain yourself or output anything else. YOU MUST BE CREATIVE!
"""

In [ ]:
stage2_prompt = """
You have been assigned a text written task: {}
Your mission is to write 30 specific text rewritten prompts for this task in JSON format. The JSON object must contain the following keys: 
- "rewrite_prompt": a string, the prompt specified by the classification task. 
Please adhere to the following guidelines: 
- The "rewrite_prompt" should be less than 20 words and diverse in expression. 
- The values for all fields should be in English. 
- The "rewrite_prompt" is clear and requires high school level education to comprehend. 
- The original text should named "text" in "rewrite_prompt"
Your output must always be a JSON object only, do not explain yourself or output anything else. YOU MUST BE CREATIVE!
"""

In [ ]:
stage3_prompt = """
I will give you an input that contains many rewrite prompts. Find all of them and return only the content of the prompt as a Python list.

Here is an output format example:
- xxxxxxxxxx.
- xxxxxxxx.

This is the input:
{}
Now finish the task I give to you.
"""

In [ ]:
topic_list = get_response(stage1_prompt)
print(topic_list)

In [ ]:
topic_list = topic_list.split("\n")
topic_list

In [ ]:
prompt_lists = []

for idx, topic in tqdm(enumerate(topic_list), total=len(topic_list)):
    clean_topic = stage2_prompt.format(topic[2:])
    json_output = get_response(clean_topic)
    print(json_output)

    json_input = stage3_prompt.format(json_output)
    final_output = get_response(json_input)
    print(final_output)

    final_output_lists = final_output.split("\n")
    for final_prompt in final_output_lists:
        prompt_lists.append(final_prompt[2:])

In [ ]:
print(prompt_lists)

In [ ]:
print(len(prompt_lists))